In [29]:
import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image as Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [30]:
img = Image.open('/kaggle/input/psut-ds-24hr-level2/Train_DS/Train_DS/mild/mild_523.jpg')

In [31]:
# get width and height
width = img.width
height = img.height
  
# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)

The height of the image is:  128
The width of the image is:  128


# Prepare the Data

In [32]:
train_dir = "/kaggle/input/psut-ds-24hr-level2/Train_DS/Train_DS"
test_dir = "/kaggle/input/psut-ds-24hr-level2/Test_DS"

In [33]:
batch_size = 32
img_height = 128
img_width = 128

In [34]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 4479 files belonging to 4 classes.
Using 3584 files for training.


In [35]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  label_mode='categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 4479 files belonging to 4 classes.
Using 895 files for validation.


In [36]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  validation_split=None,
  subset=None,
  seed=42,
  shuffle=False,
  label_mode='categorical',
  image_size=(img_height, img_width)
)

Found 1921 files belonging to 1 classes.


In [37]:
class_names = train_ds.class_names
print(class_names)

['mild', 'moderate', 'non', 'verymild']


In [38]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 128, 128, 3)
(32, 4)


In [39]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Build the Model

In [40]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [41]:
model.compile(
  optimizer=tf.optimizers.SGD(lr=0.001),
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy'])

In [42]:
class_weight = {0: 0.00159489633, 1: 0.02272727272, 2: 0.00044642857, 3: 0.0006377551}

In [44]:
history = model.fit(train_ds, validation_data=val_ds, epochs=30, class_weight=class_weight)

Epoch 1/30
112/112 [==============================] - 58s 511ms/step - loss: 1.4786 - accuracy: 0.4484 - val_loss: 1.0920 - val_accuracy: 0.4223
Epoch 2/30
112/112 [==============================] - 57s 507ms/step - loss: 1.2018 - accuracy: 0.4972 - val_loss: 0.9373 - val_accuracy: 0.5754
Epoch 3/30
112/112 [==============================] - 57s 506ms/step - loss: 1.0817 - accuracy: 0.5187 - val_loss: 1.2358 - val_accuracy: 0.3911
Epoch 4/30
112/112 [==============================] - 57s 508ms/step - loss: 1.0144 - accuracy: 0.5488 - val_loss: 0.8943 - val_accuracy: 0.5955
Epoch 5/30
112/112 [==============================] - 57s 514ms/step - loss: 0.9451 - accuracy: 0.5672 - val_loss: 0.8567 - val_accuracy: 0.5933
Epoch 6/30
112/112 [==============================] - 57s 513ms/step - loss: 0.8869 - accuracy: 0.5957 - val_loss: 0.8397 - val_accuracy: 0.6034
Epoch 7/30
112/112 [==============================] - 57s 508ms/step - loss: 0.8258 - accuracy: 0.6244 - val_loss: 0.8620 - val_ac

In [348]:
model.save('alexnet_class_weights')

# Predict Test

In [46]:
TEST_DIR_PATH = '/kaggle/input/psut-ds-24hr-level2/Test_DS/Test_DS'
files = os.listdir(TEST_DIR_PATH)
Ids = list()

for file in files:
    img_id = os.path.splitext(file)[0]
    Ids.append(img_id)

submission = pd.DataFrame({"file_name": files, "Id": Ids, "Predicted": Ids})

In [48]:
submission.head()

,file_name,Id,Predicted
0,1269.jpg,1269,1269
1,623.jpg,623,623
2,764.jpg,764,764
3,1700.jpg,1700,1700
4,1786.jpg,1786,1786


In [49]:
mask = submission[submission.file_name == "processdata.ipynb"].index
submission = submission.drop(mask)

In [50]:
submission.head()

,file_name,Id,Predicted
0,1269.jpg,1269,1269
1,623.jpg,623,623
2,764.jpg,764,764
3,1700.jpg,1700,1700
4,1786.jpg,1786,1786


In [51]:
preds = list()
for index, row in submission.iterrows():
    image_path = TEST_DIR_PATH + '/' + row['file_name']
    
    image = tf.keras.utils.load_img(image_path)
    input_arr = tf.keras.utils.img_to_array(image)
    input_arr = np.array([input_arr])
    
    pred = model.predict(input_arr)
    pred = np.argmax(pred, axis=1)
    
    preds.append(pred[0])

In [52]:
submission['Predicted'] = preds

In [53]:
label_dict = {0: 'mild', 1:'moderate', 2:'non', 3:'verymild'}

In [54]:
submission['Predicted'] = submission['Predicted'].map(label_dict)

In [55]:
submission

,file_name,Id,Predicted
0,1269.jpg,1269,non
1,623.jpg,623,non
2,764.jpg,764,non
3,1700.jpg,1700,non
4,1786.jpg,1786,verymild
...,...,...,...
1917,1801.jpg,1801,verymild
1918,147.jpg,147,non
1919,921.jpg,921,non
1920,728.jpg,728,non


In [56]:
submission['Predicted'].value_counts()

non         1405
verymild     411
mild         104
moderate       1
Name: Predicted, dtype: int64

In [54]:
submission[['Id', 'Predicted']].to_csv('submission_alexnet_classweights.csv', index=False)